<a href="https://colab.research.google.com/github/Fatimah007/T5/blob/main/Copy_of_RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [ ]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.70MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [ ]:
!pip install pandas
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [ ]:
import os

In [ ]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [ ]:
!pip install markdown
import markdown

In [ ]:
for i in range(len(df)):

    violation = df['المخالفة'].iloc[i]
    duty = df['الغرامة'].iloc[i]

    markdown_content = f"# {violation}\n\n"
    markdown_content += f"{duty}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [ ]:
# read markdownfiles

markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

In [ ]:
markdown_texts

['<h1>ترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>',
 '<h1>استعمال المكابح (الفرامل) بشكل مفاجئ لغير ضرورة.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>',
 '<h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>',
 '<h1>قيادة المركبة بالاتجاه المعاكس لحركة السير.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 '<h1>الوقوف على خطوط السكة الحديدية.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>نقل عدد من الركاب يزيد على العدد المحدد في رخصة السير.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>استخدام السائق بيده أي جهاز محمول أثناء سير المركبة.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>',
 '<h1>تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 '<h1>عدم قيام أصحاب الحيوانات بإبعادها عن حرم الطريق المعتمد.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 '<h1>طمس لوحا

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [ ]:
!pip install --upgrade langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=120, chunk_overlap=20)
documents = text_splitter.create_documents(markdown_texts)

In [ ]:
documents

[Document(metadata={}, page_content='<h1>ترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>'),
 Document(metadata={}, page_content='<h1>استعمال المكابح (الفرامل) بشكل مفاجئ لغير ضرورة.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>'),
 Document(metadata={}, page_content='<h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم'),
 Document(metadata={}, page_content='عدم وجود لوحات تنظم ذلك.</h1>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 500 - 900 ريال</p>'),
 Document(metadata={}, page_content='<h1>قيادة المركبة بالاتجاه المعاكس لحركة السير.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>'),
 Document(metadata={}, page_content='<h1>الوقوف على خطوط السكة الحديدية.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>'),
 Document(metadata={}, page_content='<h1>نقل عدد من الركاب يزيد على العدد المحدد في رخصة السير.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>'),
 Document(metada

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [ ]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
!pip install -U langchain-community

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="AbderrahmanSkiredj1/Arabic_text_embedding_for_sts")

In [ ]:
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

In [ ]:
db

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [ ]:
!pip install langchain
from langchain import PromptTemplate

In [ ]:
PROMPT_TEMPLATE="""
أجب عن السؤال حسب السياق مع إقتراح نصيحه باللغة العربية:
السياق: {context}
السؤال: {question}
الجواب:
"""
#Answer the question based only on the following context and give advice
prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
!pip install langchain_groq

In [ ]:
!pip install ChatGroq

ERROR: Could not find a version that satisfies the requirement ChatGroq (from versions: none)
ERROR: No matching distribution found for ChatGroq


In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import LLMChain

In [ ]:
groq_api_key = "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"
llm = ChatGroq(temperature=0.1, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [ ]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
def query_rag(query: str):
    similarity_results = db.similarity_search_with_score(query, k=1)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [ ]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")

Prompt after formatting:

أجب عن السؤال حسب السياق مع إقتراح نصيحه باللغة العربية:
السياق: <h1>قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>
السؤال: ماهي الغرامة على القيادة بدون رخصة؟
الجواب:


> Finished chain.


In [ ]:
print(f'Question:\n{response["question"]}\n\nAnswer: \n{response["text"]}')

Question:
ماهي الغرامة على القيادة بدون رخصة؟

Answer: 
الجواب: الغرامة المالية على القيادة بدون رخصة هي 1000 - 2000 ريال.

نصيحة: قبل القيادة، يجب عليك أن تحقق من توافر التجهيزات اللازمة مثل المكابح والأنوار وما في حكمهما، لضمان سلامةك وسلامة otros على الطريق. كما أن الحصول على رخصة القيادة هو مطلب قانوني، لذا يجب عليك أن تحصل عليها قبل القيادة.
